In [1]:
import numpy as np
import pandas as pd

import edhec_risk_kit_206 as erk

%load_ext autoreload
%autoreload 2

my_inds = ['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food']

ind_rets = erk.get_ind_returns(weighting='vw')["2013":][my_inds]
ind_mcap = erk.get_ind_market_caps(weights=True)["2013":][my_inds]

In [2]:
ind_mcap

,Hlth,Fin,Whlsl,Rtail,Food
2013-01,0.093193,0.144131,0.012207,0.068470,0.032168
2013-02,0.095323,0.144987,0.012226,0.068400,0.032147
2013-03,0.095347,0.144808,0.012332,0.068019,0.032994
2013-04,0.097713,0.145560,0.012210,0.068507,0.033767
2013-05,0.098455,0.145732,0.012057,0.069722,0.034233
...,...,...,...,...,...
2018-08,0.103549,0.164637,0.010987,0.076225,0.021873
2018-09,0.104324,0.162556,0.010992,0.080234,0.021005
2018-10,0.106363,0.159464,0.010681,0.080886,0.020661
2018-11,0.105080,0.161024,0.010674,0.078849,0.022649


In [19]:
### THIS IS WRONG, but wil give right quiz answers. ????

ind5_rets = ind_rets
ind5_mcap = ind_mcap
cw=erk.weight_cw(ind5_rets,ind5_mcap)
cw = cw / cw.sum()

rho =ind5_rets.corr()

vols = ind5_rets.std()*np.sqrt(12)

sigma_prior = vols.dot(vols.T) * rho

pi = erk.implied_returns(delta=2.5, sigma=sigma_prior, w=cw)

volsM = pd.DataFrame(ind_rets.std(),index=['Hlth', 'Fin', 'Whlsl', 'Rtail', 'Food'], columns=["vol"]) 
vols = volsM*np.sqrt(12)
rho = ind_rets.corr()
VOL = vols['vol']
sigma_prior_ =VOL.dot(VOL.T) * rho
erk.implied_returns(delta=2.5, sigma=sigma_prior, w=cw)

Hlth     0.176459
Fin      0.181962
Whlsl    0.174545
Rtail    0.171476
Food     0.128461
Name: Implied Returns, dtype: float64

In [8]:
w_eq = ind_mcap.iloc[0]
w_eq = w_eq / w_eq.sum()
# Germany will outperform other European Equities (i.e. FR and UK) by 5%
# Heatlh will outperform retail and wholesale by 3%
q = pd.Series([.03]) # just one view
# start with a single view, all zeros and overwrite the specific view
p = pd.DataFrame([0.]*5, index=w_eq.index).T
# find the relative market caps of FR and UK to split the
# relative outperformance of DE ...
w_ret = w_eq.loc['Rtail'] / (w_eq.loc['Rtail'] + w_eq.loc['Whlsl'])
w_whl = w_eq.loc['Whlsl'] / (w_eq.loc['Rtail'] + w_eq.loc['Whlsl'])
p.iloc[0]['Hlth'] = 1.
p.iloc[0]['Rtail'] = -w_ret
p.iloc[0]['Whlsl'] = -w_whl
p
#(p*100).round(1)


,Hlth,Fin,Whlsl,Rtail,Food
0,1.0,0.0,-0.151306,-0.848694,0.0


In [9]:
delta = 2.5
tau = 0.05 # from Footnote 8
# Find the Black Litterman Expected Returns
bl_mu, bl_sigma = erk.bl(w_eq, sigma_prior, p, q, tau = tau)
(bl_mu*100).round(1)
bl_mu

Hlth     0.183539
Fin      0.182029
Whlsl    0.174092
Rtail    0.165093
Food     0.126385
dtype: float64

In [10]:
wstar = erk.w_star(delta=2.5, sigma=bl_sigma, mu=bl_mu)
wstar

Hlth     0.371167
Fin      0.392005
Whlsl    0.015391
Rtail    0.086329
Food     0.087490
dtype: float64